In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, log_loss
import numpy as np
import pandas as pd
import nltk
from sklearn.neighbors import KNeighborsClassifier

C:\Users\tetex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Charger le fichier CSV
file_path = '../data/Sample_Books_rating.csv'
data = pd.read_csv(file_path)

# Afficher les premières lignes pour comprendre la structure
data.head()

,Id,Titre,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte
0,B0006CR6U4,"A dictionary of the Targumim, the Talmud Babli...",NaN,A303XPDO694V6X,Ariel,2/6,4.0,1122163200,Jastrow,Jastrow made a great workthis dictionary can h...
1,0897166159,Espresso Coffee: Professional Techniques,NaN,A3780H4TM9RMB8,David barnes,0/1,2.0,1356912000,NOT the book,Extremely disappointed by the SHORT length and...
2,0736693408,The First King of Shannara (The Sword of Shann...,NaN,A1AX6VPDQQZDPV,M Carlton,4/4,5.0,1105574400,Great (what do you expect?),"This, like all of Brook's Shannara series book..."
3,0395051029,Wuthering Heights (Riverside editions),NaN,A35RQKCCCQ62O0,LadyJ,0/0,4.0,1353888000,Satisfied,I enjoyed this classic. I didn't know the stor...
4,4770016050,"A Cat, a Man, and Two Women (Japans Modern Wri...",NaN,A2IJQDE1I4SIJT,"David C. Arnold ""master D""",1/2,5.0,1167955200,"Ordered 09/02/2006, still on backorder",I would love to read this book. Have accepted ...


In [3]:
# Prétraitement des données
data['Polarité'] = data['revue/score'].apply(lambda x: 'Positif' if x > 3 else ('Négatif' if x < 3 else 'Neutre'))
data = data[['revue/texte', 'Polarité', 'revue/score']].dropna()

data.head()

,revue/texte,Polarité,revue/score
0,Jastrow made a great workthis dictionary can h...,Positif,4.0
1,Extremely disappointed by the SHORT length and...,Négatif,2.0
2,"This, like all of Brook's Shannara series book...",Positif,5.0
3,I enjoyed this classic. I didn't know the stor...,Positif,4.0
4,I would love to read this book. Have accepted ...,Positif,5.0


In [4]:
# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(data['revue/texte'], data['Polarité'], test_size=0.2, random_state=42)
# Encodage de la polarité pour la classification
le = LabelEncoder()
y_train_polarity = le.fit_transform(y_train)
y_test_polarity = le.transform(y_test)

C:\Users\tetex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\tetex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\tetex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instea

In [5]:
list_to_avoid = nltk.corpus.stopwords.words('english') + ['great', 'books', 'quot', 'life', 'read', 'time', 'story', 'one', 'book', 'like']

In [6]:
# Représentation de texte - Mots simples
vect_simple = CountVectorizer(stop_words=list_to_avoid)
X_train_simple = vect_simple.fit_transform(X_train)
X_test_simple = vect_simple.transform(X_test)

# Représentation de texte - TF-IDF
vect_tfidf = TfidfVectorizer(stop_words=list_to_avoid)
X_train_tfidf = vect_tfidf.fit_transform(X_train)
X_test_tfidf = vect_tfidf.transform(X_test)

In [7]:
# Modélisation - Exemple avec RandomForest pour la polarité
model_rfc = RandomForestClassifier()
model_rfc.fit(X_train_simple, y_train_polarity)
y_pred1 = model_rfc.predict(X_test_simple)

model_rfc2 = RandomForestClassifier()
model_rfc2.fit(X_train_tfidf, y_train_polarity)
y_pred2 = model_rfc2.predict(X_test_tfidf)

In [8]:
# Remarque: L'entraînement et l'évaluation des modèles ne sont pas inclus dans ce code.
# évaluation des modèles accuracy_score et log_loss
print("Accuracy mots simples:",accuracy_score(y_test_polarity, y_pred1))
print("Loss mots simples:",log_loss(y_test_polarity, model_rfc.predict_proba(X_test_simple)))

print("Accuracy Tf-Idf:",accuracy_score(y_test_polarity, y_pred2))
print("Loss Tf-Idf:",log_loss(y_test_polarity, model_rfc2.predict_proba(X_test_tfidf)))

Accuracy mots simples: 0.808
Loss mots simples: 0.6684265211444668
Accuracy Tf-Idf: 0.804
Loss Tf-Idf: 0.5891608024544377
